In [ ]:
#from handcalcs.install_templates import install_html

#install_html('full_html_noinputs.tpl')

In [3]:
from handcalcs import render 
import forallpeople as si
from scipy import integrate

si.environment('default', top_level=True)

from scipy.optimize import fsolve
from handcalcs.decorator import handcalc
import sympy as sp

#def equation(x):
#    return 4**x + 6**x - 9**x
#fsolve(equation,x0=[0,10])

### Enunciado - Exercício de Conformação a Frio

Uma chapa de alumínio (σe = 30 |ϕ|^0,30 kgf/mm2) de espessura 3,5 mm e
largura 700 mm é laminada até 1,5 mm de espessura. Desprezando o alargamento, considerando μ = 0,08 (cilindros de aço
contra alumínio) e sabendo que se usa cilindros de diâmetro 25% maior que o
mínimo necessário e que os cilindros giram a 120 RPM, calcule as forças e a potências ideais de laminação.



---
Como primeira etapa, deve-se calcular a deformação verdadeira realizada durante o processo. Caso seja maior que o expoente $n$ (no caso, $0.3$), deve-se divir o processo em mais de uma etapa, uma vez que ultrapassando este limite a deformação não se dá mais de forma homogênea.

In [4]:
import numpy as np
from numpy import log as ln
from math import ceil as Arredondar
from math import fabs as Absoluto

In [5]:
%%render 
#long

n = 0.3 #máximo de deformação homogênea por passe

phi = ln(1.5/3.5) #$\phi$ é maior que $n$, necessária divisão do processo

n_passes = Arredondar(Absoluto(phi/n)) #número de passes necessários para que a deformação seja homogênea

phi_passes = phi/n_passes #valor da deformação verdadeira na espessura em cada passe

<IPython.core.display.Latex object>

---
Feito isto, se faz necessária a identificação da espessura obtida em cada processo:

In [6]:
from numpy import e

In [7]:
%%render 
#short

h_i = 3.5 * mm 

h_1 = h_i*e**phi_passes 

h_2 = h_1*e**phi_passes 

h_3 = h_2*e**phi_passes 

h_f = h_3

<IPython.core.display.Latex object>

---
Importante frisar que entre todos os passes o processo de recozimento foi realizado.

---
Agora, deve-se calcular as variações de espessura entre as passagens pelos rolos, para então determinar o raio mínimo de cada um deles:

In [8]:
%%render 
#short

Delta_h_1 = h_i - h_1

Delta_h_2 = h_1 - h_2

Delta_h_3 = h_2 - h_3

<IPython.core.display.Latex object>

---
Enfim, o raio:

In [9]:
from numpy import sin as sen
from numpy import arctan as arctg

In [10]:
%%render 
#short

mu = 0.08 #coeficiente de atrito entre o rolo e a chapa

Raio_1 = 1.25*Delta_h_1/(sen(arctg(mu))**2)

Raio_2 = 1.25*Delta_h_2/(sen(arctg(mu))**2)

Raio_3 = 1.25*Delta_h_3/(sen(arctg(mu))**2)

<IPython.core.display.Latex object>

---
Com os $Raio_i$, conseguimos calcular a largura em que a chapa entrará em contato com o rolo, e consequentente as áreas totais de contato $A_{c_i}$:

In [11]:
from math import sqrt

In [12]:
%%render short

l_d_1 = (Raio_1*Delta_h_1)**0.5

l_d_2 = (Raio_2*Delta_h_2)**0.5

l_d_3 = (Raio_3*Delta_h_3)**0.5

<IPython.core.display.Latex object>

In [13]:
%%render 
#long

b_m = 700*mm # na laminação a frio, a variação de largura é desprezível

A_c_1 = l_d_1 * b_m
A_c_2 = l_d_2 * b_m
A_c_3 = l_d_3 * b_m

<IPython.core.display.Latex object>

---
Para obter a força realizada em cada etapa, se faz necessário o cálculo da tensão de escoamento vigente durante o processo. Como o exercício propõe uma situação ideal, essa tensão nada mais é do que a média entre a tensão antes da chapa passar pelo rolo e a tensão após a chapa passar pelo rolo. 

Consegue-se obter tais valores através da equação de Holloman, que, para este caso, é:


\begin{aligned}
\sigma_{escoamento} &= \sigma_o \cdot |\phi| ^ {n} = 30 \cdot  |\phi|  ^{ 0.3 }
\end{aligned}


In [14]:
%%render 
#long

phi_plastico = 0.002 #deformação verdadeira no início da deformação plástica

sigma_o = 30*9.81*MPa #transformar de kgf/mm² para MPa

sigma_e_entrada = sigma_o*phi_plastico**n #como o material é recozido, a tensão de entrada é sempre a mesma

sigma_e_saida = sigma_o*Absoluto(phi_passes)**0.3 #como $\phi$ é constate durante as etapas, a tensão de saída também é sempre a mesma

sigma_medio = (sigma_e_entrada + sigma_e_saida)/2

<IPython.core.display.Latex object>

---
Finalmente, torna-se possível o cálculo da força ideal:

In [15]:
%%render
#long

Forca_ideal_1 = sigma_medio * A_c_1

Forca_ideal_2 = sigma_medio * A_c_2

Forca_ideal_3 = sigma_medio * A_c_3

<IPython.core.display.Latex object>

---
O cálculo do momento ideal é a última etapa. Poderemos obter a potência ideal com esta grandeza:

In [16]:
%%render
#long

Momento_ideal_1 = Forca_ideal_1 * l_d_1

Momento_ideal_2 = Forca_ideal_2 * l_d_2

Momento_ideal_3 = Forca_ideal_3 * l_d_3

<IPython.core.display.Latex object>

In [17]:
from numpy import pi 

In [18]:
%%render
#long

Rotacao = 120#RPM, definida pelo enunciado como igual para todos rolos

<IPython.core.display.Latex object>

In [19]:
%%render
#long

Rotacao = 120*2*pi/60*1/s

Potencia_ideal_1 = Momento_ideal_1 * Rotacao # ou ~ $215 cv$
Potencia_ideal_2 = Momento_ideal_2 * Rotacao # ou ~ $122 cv$
Potencia_ideal_3 = Momento_ideal_3 * Rotacao # ou ~ $69 cv$

<IPython.core.display.Latex object>

# Segunda etapa

O primeiro passo, para sair do caso ideal para o caso real, é setar setar as forças ideais como entrada na equação de $R'$.

Por enquanto, tudo será feito apenas em relação ao primeiro passo:

### Primeiro Passo

In [20]:
%%render 
#long

c = ((4.6*10**(-4))/9.81)*(mm**2)/N # constante do aço transformada para de kgf para N

Rlinha_1_1 = Raio_1*(1+((c*Forca_ideal_1)/(b_m*Delta_h_1)))

<IPython.core.display.Latex object>

In [21]:
%%render
#long

Curva_empírica = mu * sqrt(Rlinha_1_1/h_1)

epsilon = Delta_h_1/h_i

<IPython.core.display.Latex object>

Da interesecção da curva empírica com o eixo das absissas $\epsilon$, retiramos o primeiro fator de correção da forca, $f_1=1.1$.

O próximo passo é calcular a primeira força corrigida $F_1$:

In [ ]:
%%render
#long

f_1_1 = 1.1

F_1_1 = b_m*sqrt(Rlinha_1_1*Delta_h_1)*1.15*mm*sigma_medio*f_1_1

Como a $F_1$ é muito maior que a $Forca_{ideal_1}$, se faz necessária a repetição do processo:

In [ ]:
%%render
#long

Rlinha_1_2 = Raio_1*(1+((c*F_1_1)/(b_m*Delta_h_1)))

In [ ]:
%%render
#long

Curva_empírica = mu * sqrt(Rlinha_1_2/h_1)

epsilon = Delta_h_1/h_i

Os valores foram extremamente parecidos. Provavelmente o resultado irá convergir:

In [ ]:
%%render
#long

f_1_2 = 1.15

F_1_2 = b_m*sqrt(Rlinha_1_2*Delta_h_1)*1.15*mm*sigma_medio*f_1_2

In [ ]:
print('A diferença entre as forças F_2 e F_1 é {:+.3%}, podendo ser considerada baixa o suficiente para parar a iteração.'.format(F_1_2/F_1_1 - 1))

O próximo passo consiste na obtenção do outro fator de correção, $f_2$, com as mesmas credenciais só que em outra figura.

Sabendo que

In [ ]:
%%render
#parameters

Curva_empírica = mu * sqrt(Rlinha_1_2/h_1)

epsilon = Delta_h_1/h_i

Obtém-se como $f_2$ o valor de 0.1, e calcula-se o Torque corrigido:

In [ ]:
%%render
#long

f_1_momento = 0.1

Momento_corrigido_1 = 2*Raio_1*(((h_i)**2)/h_1)*sigma_medio*b_m*f_1_momento

Potencia_corrigida_1 = Momento_corrigido_1*Rotacao

### Segundo Passo

Todo o processo é repetido.

In [ ]:
%%render
#long

Rlinha_2_1 = Raio_2*(1+((c*Forca_ideal_2)/(b_m*Delta_h_2)))

In [ ]:
%%render
#long

Curva_empírica = mu * sqrt(Rlinha_2_1/h_2)

epsilon = Delta_h_2/h_1 #este cálculo representa o quanto variou em relação ao comprimento inicial!

Novamente, $f$ é obtido como $1.1$:

In [ ]:
%%render
#long

f_2_1 = 1.1

#rlinha1 é relacionado ao passe, deltah2 é relacionado ao exercício inteiro
#ou seja, o primeiro cálculo desse passe sempre vai apresentar rlinha1,
#e, caso esse passo seja o segundo, o delta sempre vai ser o 2.

F_2_1 = b_m*sqrt(Rlinha_2_1*Delta_h_2)*1.15*mm*sigma_medio*f_2_1

In [ ]:
%%render
#long

Rlinha_2_2 = Raio_2*(1+((c*F_2_1)/(b_m*Delta_h_2)))

In [ ]:
%%render
#long

Curva_empírica = mu * sqrt(Rlinha_2_2/h_2)

epsilon = Delta_h_2/h_1

In [ ]:
%%render
#Long

f_2_2 = 1.15 #retirado do gráfico

F_2_2 = b_m*sqrt(Rlinha_2_2*Delta_h_2)*1.15*mm*sigma_medio*f_2_2

In [ ]:
%%render
#Long

f_2_momento = 0.09

Momento_corrigido_2 = 2*Raio_2*(((h_1)**2)/(h_2))*sigma_medio*b_m*f_2_momento

Potencia_corrigida_2 = Momento_corrigido_2 * Rotacao

### Terceiro passo

In [ ]:
%%render
#long

Rlinha_3_1 = Raio_3*(1+((c*Forca_ideal_3)/(b_m*Delta_h_3)))

In [ ]:
%%render
#long

Curva_empírica = mu * sqrt(Rlinha_3_1/h_3)

epsilon = Delta_h_3/h_2 #este cálculo representa o quanto variou em relação ao comprimento inicial!

In [ ]:
%%render
#long

f_3_1 = 1.1

#rlinha1 é relacionado ao passe, deltah2 é relacionado ao exercício inteiro
#ou seja, o primeiro cálculo desse passe sempre vai apresentar rlinha1,
#e, caso esse passo seja o segundo, o delta sempre vai ser o 2.

F_3_1 = b_m*sqrt(Rlinha_3_1*Delta_h_3)*1.15*mm*sigma_medio*f_3_1

In [ ]:
%%render
#long

Rlinha_3_2 = Raio_3*(1+((c*F_3_1)/(b_m*Delta_h_3)))

In [ ]:
%%render
#long

Curva_empírica = mu * sqrt(Rlinha_3_2/h_3)

epsilon = Delta_h_3/h_2 #este cálculo representa o quanto variou em relação ao comprimento inicial!

In [ ]:
%%render
#long

f_3_2 = 1.1

#rlinha1 é relacionado ao passe, deltah2 é relacionado ao exercício inteiro
#ou seja, o primeiro cálculo desse passe sempre vai apresentar rlinha1,
#e, caso esse passo seja o segundo, o delta sempre vai ser o 2.

F_3_2 = b_m*sqrt(Rlinha_3_2*Delta_h_3)*1.15*mm*sigma_medio*f_3_2

In [ ]:
%%render
#long

f_3_momento = 0.09

Momento_corrigido_3 = 2*Raio_3*(((h_2)**2)/(h_3))*sigma_medio*b_m*f_3_momento

Potencia_corrigida_3 = Momento_corrigido_3 * Rotacao


In [ ]:
import pandas as pd

resultados = pd.DataFrame(index=['Força','Momento','Potência'])

resultados.style.set_caption("Comparação Ideal x Real das Grandezas")

resultados['Etapa 1'] = [(Forca_ideal_1, F_1_2),(Momento_ideal_1, Momento_corrigido_1),(Potencia_ideal_1, Potencia_corrigida_1)]
resultados['Etapa 2'] = [(Forca_ideal_2, F_2_2),(Momento_ideal_2, Momento_corrigido_2),(Potencia_ideal_2, Potencia_corrigida_2)]
resultados['Etapa 3'] = [(Forca_ideal_3, F_3_2),(Momento_ideal_3, Momento_corrigido_3),(Potencia_ideal_3, Potencia_corrigida_3)]


resultados